In [1]:
import sys
import os
sys.path.append(sys.path[0].replace('/notebooks',''))

In [2]:
from dataset import DatasetGeneral
import torch
from torch import nn
from torchvision.models import mobilenet_v2
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, InterpolationMode, Resize, CenterCrop, Normalize, PILToTensor, ConvertImageDtype, ToTensor

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import cv2
from PIL import Image

from tqdm import tqdm

import os

In [3]:
model = mobilenet_v2(weights='IMAGENET1K_V1')
model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [4]:
model = model.train()

In [5]:
from utils import get_images_labels
from sklearn.model_selection import train_test_split

In [6]:
df = get_images_labels("/home/lumalfa/datasets/DigiFace-1M/",format_img='png')

Images and labels loaded


In [7]:
map_labels = {j:i for i,j in enumerate(df['images_labels'].unique())}
df['images_labels'] = df['images_labels'].apply(lambda x: map_labels[x])

In [8]:
train, aux = train_test_split(df,test_size=0.3,random_state=69,stratify=df['images_labels'])
val, test = train_test_split(aux,test_size=0.5,random_state=69,stratify=aux['images_labels'])

In [9]:
workers = 0
batch_size = 2

transform = Compose([
        Resize(256),
        CenterCrop(224),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    ])

print("------- Starting dataloaders -------")

train_dataset   = DatasetGeneral(train['images_paths'].tolist(), train['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)
val_dataset     = DatasetGeneral(val['images_paths'].tolist(), val['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)
test_dataset    = DatasetGeneral(test['images_paths'].tolist(), test['images_labels'].tolist(), transform_data=transform,pos_neg_examples=4)

train_dataloader = DataLoader(train_dataset, num_workers=workers, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, num_workers=workers, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, num_workers=workers, batch_size=batch_size)

------- Starting dataloaders -------


In [10]:
triplet_loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)

In [24]:
model = model.train()
for anchor, positive, negative in tqdm(train_dataloader,total=train_dataloader.__len__()):
    anchor_pred = model(anchor)
    positive_pred = [model(x) for x in positive]
    negative_pred = [model(x) for x in negative]
    
    loss = triplet_loss(anchor_pred,positive_pred,negative_pred)
    loss.backward()
    break

  0%|                                                                                         | 0/50400 [00:00<?, ?it/s]/home/lumalfa/Face-Similarity/dataset.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(anchor), torch.stack(positive), torch.stack(negative)
  0%|                                                                                         | 0/50400 [00:11<?, ?it/s]


TypeError: triplet_margin_loss(): argument 'positive' (position 2) must be Tensor, not list

In [38]:
torch.stack(positive_pred)[0].shape

torch.Size([4, 1280, 7, 7])

In [45]:
anchor_pred[0].unsqueeze(0).shape

torch.Size([1, 1280, 7, 7])

In [13]:
positive.shape

torch.Size([2, 4, 3, 224, 224])

In [19]:
positive.unsqueeze(0).shape

torch.Size([1, 2, 4, 3, 224, 224])

In [10]:
anchor = torch.rand(100,1,1028)
positive = torch.rand(100,10,1028)
negative = torch.rand(100,10,1028)

In [33]:
anchor_pred.unsqueeze(0).shape

torch.Size([1, 2, 1280, 7, 7])

In [29]:
triplet_loss(anchor_pred.unsqueeze(0),torch.stack(positive_pred),torch.stack(negative_pred))

RuntimeError: The size of tensor a (2) must match the size of tensor b (4) at non-singleton dimension 1